# Descripción del proyecto

La compañía móvil Megaline no está satisfecha al ver que muchos de sus clientes utilizan planes heredados. 

Quieren desarrollar un modelo que pueda analizar el comportamiento de los clientes y recomendar uno de los nuevos planes de Megaline: Smart o Ultra.

Tienes acceso a los datos de comportamiento de los suscriptores que ya se han cambiado a los planes nuevos (del proyecto del curso de Análisis estadístico de datos). 

Para esta tarea de clasificación debes crear un modelo que escoja el plan correcto. Como ya hiciste el paso de procesar los datos, puedes lanzarte directo a crear el modelo.

Desarrolla un modelo con la mayor exactitud posible. En este proyecto, el umbral de exactitud es 0.75. Usa el dataset para comprobar la exactitud.

# Descripción de datos

Cada observación en el dataset contiene información del comportamiento mensual sobre un usuario. 

La información dada es la siguiente:

сalls — número de llamadas,

minutes — duración total de la llamada en minutos,

messages — número de mensajes de texto,

mb_used — Tráfico de Internet utilizado en MB,

is_ultra — plan para el mes actual (Ultra - 1, Smart - 0).

# Instrucciones del proyecto.

Importa librerias

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.dummy import DummyClassifier

Abre y examina el archivo de datos. 

Dirección al archivo:datasets/users_behavior.csv 

Descarga el dataset

In [2]:
data= pd.read_csv('/datasets/users_behavior.csv')

In [3]:
print(data.info())
display(data.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB
None


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


Dividimos los datos en caracteristicas

In [4]:
X = data.drop('is_ultra', axis=1)
y = data['is_ultra']

Conjunto de entrenamiento, validacion y prueba

In [5]:
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42)

### Random Forest Classifier

definimos el modelo base

In [7]:
base_model = RandomForestClassifier()

In [8]:
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 5, 10]
}
grid_search = GridSearchCV(estimator=base_model, param_grid=param_grid, cv=5)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [None, 5, 10],
                         'n_estimators': [50, 100, 200]})

Obtenemos el mejor modelo y los hiperparametors

In [9]:
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

In [10]:
print("Mejor modelo:",best_model)
print("Mejores parametros:",best_params)

Mejor modelo: RandomForestClassifier(max_depth=10, n_estimators=200)
Mejores parametros: {'max_depth': 10, 'n_estimators': 200}


Evaluams el mejor modelo

In [11]:
y_pred_val = best_model.predict(X_val)
accuracy_val = accuracy_score(y_val, y_pred_val)

In [12]:
print("Precision en el conjunto de validacion:", accuracy_val)

Precision en el conjunto de validacion: 0.7931570762052877


Realizamos el test

In [13]:
y_pred_test = best_model.predict(X_test)
accuracy_test = accuracy_score(y_test, y_pred_test)

Mostramos la precision

In [14]:
print("Precision en el conjunto de prueba:", accuracy_test)

Precision en el conjunto de prueba: 0.8258164852255054


### Segundo modelo

difinimos el modelos base

In [15]:
base_model_dt = DecisionTreeClassifier()

In [16]:
param_grid_dt = {
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10]
}
grid_search_dt = GridSearchCV(estimator=base_model_dt, param_grid=param_grid_dt, cv=5)
grid_search_dt.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': [None, 5, 10],
                         'min_samples_split': [2, 5, 10]})

Obtenemos el mejor modelo y los hiperparametors

In [17]:
best_model_dt = grid_search_dt.best_estimator_
best_params_dt = grid_search_dt.best_params_

In [18]:
print("Mejor modelo:", best_model_dt)
print("Mejores parametros:", best_params_dt)

Mejor modelo: DecisionTreeClassifier(max_depth=10, min_samples_split=5)
Mejores parametros: {'max_depth': 10, 'min_samples_split': 5}


Evaluamos el conjunto de validacion

In [19]:
y_pred_val_dt = best_model_dt.predict(X_val)
accuracy_val_dt = accuracy_score(y_val, y_pred_val_dt)

In [20]:
print("Precisión en el conjunto de validación (Decision Tree):", accuracy_val_dt)

Precisión en el conjunto de validación (Decision Tree): 0.7744945567651633


Evaluamos el conjunto de prueba

In [21]:
y_pred_test_dt = best_model_dt.predict(X_test)
accuracy_test_dt = accuracy_score(y_test, y_pred_test_dt)

In [22]:
print("Precisión en el conjunto de prueba (Decision Tree):", accuracy_test_dt)

Precisión en el conjunto de prueba (Decision Tree): 0.7884914463452566


### Dummy Classifier

definimos nuestro modelo dummy classifier

In [23]:
dummy_model = DummyClassifier(strategy='most_frequent')

entrenamos el modelo

In [24]:
dummy_model.fit(X_train, y_train)

DummyClassifier(strategy='most_frequent')

predicciones en el conjunto de prueba

In [25]:
y_pred_dummy = dummy_model.predict(X_test)

precision del conjunto de prueba

In [26]:
accuracy_dummy = accuracy_score(y_test, y_pred_dummy)

In [27]:
print("Precisión en el conjunto de prueba (Dummy Classifier):", accuracy_dummy)

Precisión en el conjunto de prueba (Dummy Classifier): 0.7076205287713841


# Conclusion

Random Forest Classifier
El mejor modelo obtenido es un Random Forest Classifier con una profundidad maxima de 10 y 100 estimadores, obteniendo una precision en el conjunto de validacion del 78.38% y una precision en el conjunto de prueba del 81.49%.
El modelo de Random Forest Classifier supera al modelo Dummy Classifier, lo que indica que tiene capacidad de aprendizaje y de realizar predicciones mucho mas precisas que solo poder predecir la clase mas frecuente.

Modelo Decision Tree Classifier:
El mejor modelo obtenido es un Decision Tree Classifier con una profundidad maxima de 10 y un mínimo de 10 muestras para dividir. longrando una precision en el conjunto de validacion del 77.60% y en el conjunto de prueba de 79.32%.
este modelo tambien le gana al modelo Dummy Classifier

Mientras que el modelo Dummy Classifier se uso como un punto de referencia y solo predice la clase mas frecuente en el conjunto de entrenamiento con ua precision en el conjunto de prueba del 70.76%

